<a href="https://colab.research.google.com/github/maxim-popkov/study/blob/master/Llama_2_Fine_Tuning_using_QLora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Finetune Llama-2-7b on a Google colab

Welcome to this Google Colab notebook that shows how to fine-tune the recent Llama-2-7b model on a single Google colab and turn it into a chatbot

We will leverage PEFT library from Hugging Face ecosystem, as well as QLoRA for more memory efficient finetuning

## Setup

Run the cells below to setup and install the required libraries. For our experiment we will need `accelerate`, `peft`, `transformers`, `datasets` and TRL to leverage the recent [`SFTTrainer`](https://huggingface.co/docs/trl/main/en/sft_trainer). We will use `bitsandbytes` to [quantize the base model into 4bit](https://huggingface.co/blog/4bit-transformers-bitsandbytes). We will also install `einops` as it is a requirement to load Falcon models.

In [1]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets
!pip install -q bitsandbytes einops

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.1/88.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 76.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.

## Dataset



In [104]:
data = [
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Привет. Менеджер: Добрый день! Хотели бы вы рассмотреть наши новые предложения по кредитованию? Клиент: Сейчас не актуально для меня. \n### Answer:\n Да, есть отказ. Клиенту сейчас не актуально рассматривать предложения по кредитованию. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Какие у вас условия? Менеджер: У нас лучшие ставки на депозиты. Хотите открыть? Клиент: Подумаю и перезвоню. \n### Answer:\n Да, есть отказ. Клиент хочет подумать. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Что вы предлагаете? Менеджер: Новую программу страхования. Клиент: Уже оформил в другом банке. \n### Answer:\n Да, есть отказ. Клиент уже оформил аналогичную услугу в другом банке. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Я слышал о ваших услугах. Менеджер: Отлично! Хотели бы вы присоединиться? Клиент: Пока не решил. \n### Answer:\n Да, есть отказ. Клиент еще не принял решение. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Есть ли у вас акции? Менеджер: Да, специальное предложение на кредитные карты. Клиент: Не интересует. \n### Answer:\n Да, есть отказ. Клиенту не интересно предложение по кредитным картам. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Расскажите о ваших услугах. Менеджер: Мы предлагаем выгодные инвестиционные программы. Клиент: Не в этот раз. \n### Answer:\n Да, есть отказ. Клиенту сейчас не интересно. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Что нового? Менеджер: У нас новая программа лояльности для клиентов. Клиент: Уже слышал, спасибо. \n### Answer:\n Да, есть отказ. Клиент уже знает о данном предложении. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Какие услуги у вас есть? Менеджер: Мы предлагаем выгодные условия по ипотеке. Клиент: Спасибо, но я уже рассмотрел другие варианты. \n### Answer:\n Да, есть отказ. Клиент уже рассмотрел другие варианты. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Я ищу хорошие условия по кредиту. Менеджер: У нас есть отличное предложение для вас. Клиент: Пока рассматриваю другие банки. \n### Answer:\n Да, есть отказ. Клиент рассматривает предложения других банков. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Что вы можете предложить? Менеджер: У нас есть новая программа для молодых предпринимателей. Клиент: Звучит интересно, но сейчас не готов решить. \n### Answer:\n Да, есть отказ. Клиент сейчас не готов принять решение. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: У меня болит зуб. Менеджер: Мы можем предложить вам процедуру отбеливания зубов со скидкой. Клиент: Мне нужно лечение, а не отбеливание. \n### Answer:\n Да, есть отказ. Клиенту нужно лечение, а не отбеливание зубов. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Хочу на море в июле. Менеджер: У нас есть отличные туры в горы в августе. Клиент: Мне нужен именно морской отдых. \n### Answer:\n Да, есть отказ. Клиент хочет на море, а не в горы. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: У вас есть услуга чистки зубов? Менеджер: Да, и к тому же у нас акция на установку брекетов. Клиент: Мне только чистка нужна. \n### Answer:\n Да, есть отказ. Клиенту нужна только чистка зубов. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Я ищу тур в Таиланд. Менеджер: А как насчет тура в Индию? Это тоже очень интересное направление. Клиент: Нет, спасибо, я рассматриваю только Таиланд. \n### Answer:\n Да, есть отказ. Клиент заинтересован только в Таиланде. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: У меня проблемы с деснами. Менеджер: Мы можем предложить вам новую зубную пасту. Клиент: Мне нужен врач, а не паста. \n### Answer:\n Да, есть отказ. Клиенту нужна консультация врача. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Я бы хотел узнать о турах в Европу. Менеджер: У нас есть отличные предложения по Азии. Клиент: Я спрашивал про Европу. \n### Answer:\n Да, есть отказ. Клиент интересуется только Европой. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Мне нужно удалить зуб мудрости. Менеджер: Приходите к нам на консультацию, и мы предложим вам отбеливание зубов. Клиент: Сначала удаление, потом обсудим остальное. \n### Answer:\n Да, есть отказ. Клиенту сейчас важнее удаление зуба. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Я хочу на экскурсионный тур. Менеджер: У нас есть отличные пляжные туры. Клиент: Мне интересны экскурсии, не пляж. \n### Answer:\n Да, есть отказ. Клиенту интересны только экскурсионные туры. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: У меня чувствительные зубы. Менеджер: Мы можем предложить вам керамические вкладки. Клиент: Сначала хочу решить проблему с чувствительностью. \n### Answer:\n Да, есть отказ. Клиент хочет сначала решить проблему с чувствительностью зубов. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Я ищу тур на Новый год. Менеджер: У нас есть отличные предложения на летний отпуск. Клиент: Мне нужен именно новогодний тур. \n### Answer:\n Да, есть отказ. Клиент ищет тур именно на Новый год. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: У меня кровоточат десны. Менеджер: Мы предлагаем новую зубную нить. Клиент: Мне нужен прием у врача. \n### Answer:\n Да, есть отказ. Клиенту нужен прием у врача, а не зубная нить. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Я хочу тур в Испанию. Менеджер: А что вы думаете о Греции? Клиент: Нет, только Испания. \n### Answer:\n Да, есть отказ. Клиент хочет только в Испанию. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Мне нужна консультация ортодонта. Менеджер: У нас акция на чистку зубов! Клиент: Сначала консультация. \n### Answer:\n Да, есть отказ. Клиенту сейчас нужна консультация ортодонта. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Я ищу романтический тур для двоих. Менеджер: У нас есть групповые экскурсионные туры. Клиент: Нам нужно что-то более уединенное. \n### Answer:\n Да, есть отказ. Клиент ищет романтический тур, а не групповой. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: У меня болит корень зуба. Менеджер: Мы предлагаем новые зубные щетки. Клиент: Мне нужно лечение. \n### Answer:\n Да, есть отказ. Клиенту нужно лечение, а не новая зубная щетка. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Я хочу тур на Мальдивы. Менеджер: А как насчет Сейшел? Клиент: Только Мальдивы. \n### Answer:\n Да, есть отказ. Клиент хочет только на Мальдивы. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Мне нужно установить имплантат. Менеджер: У нас сейчас акция на отбеливание зубов. Клиент: Сначала имплантат. \n### Answer:\n Да, есть отказ. Клиенту сейчас нужен имплантат. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Я ищу тур в Европу на зиму. Менеджер: У нас есть отличные летние туры по Европе. Клиент: Мне нужен зимний тур. \n### Answer:\n Да, есть отказ. Клиент ищет зимний тур. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Мне нужна пломба. Менеджер: Мы предлагаем новые электрические зубные щетки. Клиент: Сейчас мне нужна пломба. \n### Answer:\n Да, есть отказ. Клиенту нужна пломба. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Я хочу тур в Италию. Менеджер: А что вы думаете о Франции? Клиент: Нет, меня интересует только Италия. \n### Answer:\n Да, есть отказ. Клиент хочет только в Италию. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: У меня проблемы с деснами. Менеджер: Мы предлагаем новую зубную пасту. Клиент: Мне нужен прием у врача. \n### Answer:\n Да, есть отказ. Клиенту нужен прием у врача. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Я ищу пляжный тур. Менеджер: У нас есть отличные экскурсионные туры. Клиент: Мне нужен пляж. \n### Answer:\n Да, есть отказ. Клиенту нужен пляжный тур. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Мне нужно удалить зуб. Менеджер: У нас акция на профессиональную чистку. Клиент: Сначала удаление. \n### Answer:\n Да, есть отказ. Клиенту сейчас нужно удаление зуба. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Я хочу тур в Германию. Менеджер: А как насчет Австрии? Очень близко и красиво. Клиент: Только Германия. \n### Answer:\n Да, есть отказ. Клиент хочет только в Германию. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Мне нужна консультация стоматолога. Менеджер: У нас сейчас акция на отбеливание зубов. Клиент: Сначала консультация. \n### Answer:\n Да, есть отказ. Клиенту сейчас нужна консультация стоматолога. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Я ищу тур на Канары. Менеджер: У нас есть отличные предложения на Бали. Клиент: Мне нужны именно Канары. \n### Answer:\n Да, есть отказ. Клиент хочет тур именно на Канары. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Мне нужно исправить прикус. Менеджер: Мы предлагаем новые зубные щетки. Клиент: Мне нужен ортодонт. \n### Answer:\n Да, есть отказ. Клиенту нужен ортодонт. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Я хочу тур в Францию. Менеджер: А что вы думаете о Швейцарии? Клиент: Нет, только Франция. \n### Answer:\n Да, есть отказ. Клиент хочет только в Францию. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Мне нужно установить коронку. Менеджер: У нас акция на отбеливание зубов. Клиент: Сначала коронка. \n### Answer:\n Да, есть отказ. Клиенту сейчас нужна коронка. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Мне нужен новый тариф на интернет. Менеджер: У нас есть отличные предложения на мобильную связь. Клиент: Мне нужен только интернет. \n### Answer:\n Да, есть отказ. Клиенту нужен только интернет. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: У меня проблемы с сигналом. Менеджер: Мы предлагаем новые аксессуары для телефона. Клиент: Мне нужно решить проблему с сигналом. \n### Answer:\n Да, есть отказ. Клиенту нужно решить проблему с сигналом. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Я ищу тур в Грецию. Менеджер: А как насчет тура в Турцию? Клиент: Нет, только Греция. \n### Answer:\n Да, есть отказ. Клиент хочет только в Грецию. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Мне нужен тариф с безлимитным интернетом. Менеджер: У нас есть тариф с 10 ГБ интернета в месяц. Клиент: Мне нужен безлимит. \n### Answer:\n Да, есть отказ. Клиенту нужен тариф с безлимитным интернетом. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: У меня болит зуб. Менеджер: Мы предлагаем новые зубные щетки. Клиент: Мне нужно лечение. \n### Answer:\n Да, есть отказ. Клиенту нужно лечение, а не новая зубная щетка. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Я хочу поменять свой номер телефона. Менеджер: У нас есть отличные предложения на новые смартфоны. Клиент: Мне нужно только поменять номер. \n### Answer:\n Да, есть отказ. Клиент хочет только поменять номер. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Мне нужен тариф для роуминга. Менеджер: У нас есть отличные предложения для местных звонков. Клиент: Мне нужен роуминг. \n### Answer:\n Да, есть отказ. Клиенту нужен тариф для роуминга. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Я ищу тур в Италию. Менеджер: А что вы думаете о Франции? Клиент: Нет, только Италия. \n### Answer:\n Да, есть отказ. Клиент хочет только в Италию. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Мне нужен быстрый интернет дома. Менеджер: У нас есть отличные предложения на мобильный интернет. Клиент: Мне нужен домашний интернет. \n### Answer:\n Да, есть отказ. Клиенту нужен домашний интернет. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: У меня проблемы с мобильной связью. Менеджер: Мы предлагаем новые тарифы на интернет. Клиент: Мне нужно решить проблему со связью. \n### Answer:\n Да, есть отказ. Клиенту нужно решить проблему со связью. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Мне нужен новый тариф на интернет. Менеджер: У нас есть отличные предложения на интернет. Клиент: Отлично, расскажите подробнее. \n### Answer:\n Нет отказа. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: У меня проблемы с сигналом. Менеджер: Мы можем предложить вам усилитель сигнала. Клиент: Замечательно, давайте попробуем. \n### Answer:\n Нет отказа. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Я ищу тур в Грецию. Менеджер: У нас есть отличные предложения на Грецию. Клиент: Здорово, давайте обсудим. \n### Answer:\n Нет отказа. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Мне нужен кредит на развитие бизнеса. Менеджер: У нас есть отличные условия по бизнес-кредитованию. Клиент: Прекрасно, давайте рассмотрим условия. \n### Answer:\n Нет отказа. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: У меня болит зуб. Менеджер: Мы можем предложить вам прием у стоматолога. Клиент: Хорошо, давайте запишемся. \n### Answer:\n Нет отказа. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Я хочу поменять свой номер телефона. Менеджер: Мы можем это сделать для вас. Клиент: Отлично, давайте сделаем. \n### Answer:\n Нет отказа. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Мне нужен новый тариф на интернет. Менеджер: У нас есть отличные предложения на интернет. Клиент: Отлично, расскажите подробнее. \n### Answer:\n Нет отказа. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: У меня проблемы с сигналом. Менеджер: Мы можем предложить вам усилитель сигнала. Клиент: Замечательно, давайте попробуем. \n### Answer:\n Нет отказа. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Я ищу тур в Грецию. Менеджер: У нас есть отличные предложения на Грецию. Клиент: Здорово, давайте обсудим. \n### Answer:\n Нет отказа. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Мне нужен кредит на развитие бизнеса. Менеджер: У нас есть отличные условия для предпринимателей. Клиент: Прекрасно, давайте рассмотрим условия. \n### Answer:\n Нет отказа. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: У меня болит зуб. Менеджер: Мы можем предложить вам прием у стоматолога. Клиент: Хорошо, давайте запишемся. \n### Answer:\n Нет отказа. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Я хочу поменять свой номер телефона. Менеджер: Мы можем это сделать для вас. Клиент: Отлично, давайте сделаем. \n### Answer:\n Нет отказа. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Мне нужен тариф для роуминга. Менеджер: У нас есть специальные тарифы для роуминга. Клиент: Замечательно, расскажите подробнее. \n### Answer:\n Нет отказа. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Я ищу тур в Италию. Менеджер: У нас есть отличные предложения на Италию. Клиент: Прекрасно, давайте рассмотрим варианты. \n### Answer:\n Нет отказа. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Мне нужен быстрый интернет дома. Менеджер: У нас есть отличные тарифы на домашний интернет. Клиент: Отлично, давайте выберем подходящий. \n### Answer:\n Нет отказа. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: У меня проблемы с мобильной связью. Менеджер: Мы можем предложить вам усилитель сигнала. Клиент: Звучит хорошо, давайте попробуем. \n### Answer:\n Нет отказа. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Мне нужен новый тариф на интернет. Менеджер: У нас есть отличные предложения на интернет. Клиент: Отлично, расскажите подробнее. \n### Answer:\n Нет отказа. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: У меня болит зуб. Менеджер: Мы можем предложить вам прием у стоматолога. Клиент: Хорошо, давайте запишемся. \n### Answer:\n Нет отказа. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Я ищу тур в Грецию. Менеджер: У нас есть отличные предложения на Грецию. Клиент: Здорово, давайте обсудим. \n### Answer:\n Нет отказа. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Я хочу взять кредит на развитие бизнеса. Менеджер: У нас есть отличные условия для бизнес-кредитования. Клиент: Прекрасно, давайте рассмотрим условия. \n### Answer:\n Нет отказа. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Мне нужен быстрый интернет дома. Менеджер: У нас есть отличные тарифы на домашний интернет. Клиент: Отлично, давайте выберем подходящий. \n### Answer:\n Нет отказа. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Я хочу открыть вклад. Менеджер: У нас есть выгодные условия для вкладов. Клиент: Замечательно, давайте обсудим детали. \n### Answer:\n Нет отказа. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Мне нужен тариф для роуминга. Менеджер: У нас есть специальные тарифы для роуминга. Клиент: Замечательно, расскажите подробнее. \n### Answer:\n Нет отказа. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Я ищу тур в Италию. Менеджер: У нас есть отличные предложения на Италию. Клиент: Прекрасно, давайте рассмотрим варианты. \n### Answer:\n Нет отказа. \n### End" },
{ "text": "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: У меня проблемы с мобильной связью. Менеджер: Мы можем предложить вам усилитель сигнала. Клиент: Звучит хорошо, давайте попробуем. \n### Answer:\n Нет отказа. \n### End" }
]

In [105]:
import os
os.environ["WANDB_MODE"]="offline"

In [106]:
from datasets import load_dataset
from datasets import Dataset


In [107]:
dataset = Dataset.from_list(data)

In [108]:
# from datasets import load_dataset

# #dataset_name = "timdettmers/openassistant-guanaco" ###Human ,.,,,,,, ###Assistant

# dataset_name = 'AlexanderDoria/novel17_test' #french novels
# dataset = load_dataset(dataset_name, split="train")

In [109]:
dataset[0]

{'text': '### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Привет. Менеджер: Добрый день! Хотели бы вы рассмотреть наши новые предложения по кредитованию? Клиент: Сейчас не актуально для меня. \n### Answer:\n Да, есть отказ. Клиенту сейчас не актуально рассматривать предложения по кредитованию. \n### End'}

In [110]:
len(dataset[10]['text'])

329

In [111]:
print(dataset[10]['text'])

### Instruction:
 Проанализируй и определи, есть ли отказ. Если есть, объясни, почему 
### Input:
 Клиент: У меня болит зуб. Менеджер: Мы можем предложить вам процедуру отбеливания зубов со скидкой. Клиент: Мне нужно лечение, а не отбеливание. 
### Answer:
 Да, есть отказ. Клиенту нужно лечение, а не отбеливание зубов. 
### End


In [112]:
dataset[-1]

{'text': '### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: У меня проблемы с мобильной связью. Менеджер: Мы можем предложить вам усилитель сигнала. Клиент: Звучит хорошо, давайте попробуем. \n### Answer:\n Нет отказа. \n### End'}

In [113]:
dataset

Dataset({
    features: ['text'],
    num_rows: 74
})

In [114]:
dataset.info

DatasetInfo(description='', citation='', homepage='', license='', features={'text': Value(dtype='string', id=None)}, post_processed=None, supervised_keys=None, task_templates=None, builder_name=None, dataset_name=None, config_name=None, version=None, splits=None, download_checksums=None, download_size=None, post_processing_size=None, dataset_size=None, size_in_bytes=None)

In [115]:
dataset.to_pandas()

,text
0,"### Instruction:\n Проанализируй и определи, е..."
1,"### Instruction:\n Проанализируй и определи, е..."
2,"### Instruction:\n Проанализируй и определи, е..."
3,"### Instruction:\n Проанализируй и определи, е..."
4,"### Instruction:\n Проанализируй и определи, е..."
...,...
69,"### Instruction:\n Проанализируй и определи, е..."
70,"### Instruction:\n Проанализируй и определи, е..."
71,"### Instruction:\n Проанализируй и определи, е..."
72,"### Instruction:\n Проанализируй и определи, е..."


In [116]:
dir(dataset)

['_TF_DATASET_REFS',
 '__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getitems__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_build_local_temp_path',
 '_check_index_is_initialized',
 '_data',
 '_estimate_nbytes',
 '_fingerprint',
 '_format_columns',
 '_format_kwargs',
 '_format_type',
 '_generate_tables_from_cache_file',
 '_generate_tables_from_shards',
 '_get_cache_file_path',
 '_get_output_signature',
 '_getitem',
 '_indexes',
 '_indices',
 '_info',
 '_map_single',
 '_new_dataset_with_indices',
 '_output_all_columns',
 '_push_parquet_shards_to_hub',
 '_save_to_disk_single',
 '_select_contiguous',
 '_select_w

## Loading the model

In [132]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "TinyPixel/Llama-2-7B-bf16-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

Let's also load the tokenizer below

In [133]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [134]:
tokenizer.eos_token

'</s>'

In [135]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

## Loading the trainer

Here we will use the [`SFTTrainer` from TRL library](https://huggingface.co/docs/trl/main/en/sft_trainer) that gives a wrapper around transformers `Trainer` to easily fine-tune models on instruction based datasets using PEFT adapters. Let's first load the training arguments below.

In [50]:
print(f'{0.00002:.7f}')

0.0000200


In [139]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 1
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 100
logging_steps = 10
learning_rate = 0.0002
max_grad_norm = 0.3
max_steps = 1000
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

Then finally pass everthing to the trainer

In [140]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

Map:   0%|          | 0/74 [00:00<?, ? examples/s]

We will also pre-process the model by upcasting the layer norms in float 32 for more stable training

In [141]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

## Train the model

Now let's train the model! Simply call `trainer.train()`

In [142]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,1.805400
20,0.917600
30,0.537300
40,0.441200
50,0.304700
60,0.240900
70,0.215200
80,0.180700
90,0.140800
100,0.145300


TrainOutput(global_step=1000, training_loss=0.09204635772109032, metrics={'train_runtime': 6012.0654, 'train_samples_per_second': 0.665, 'train_steps_per_second': 0.166, 'total_flos': 9626484712857600.0, 'train_loss': 0.09204635772109032, 'epoch': 54.05})

During training, the model should converge nicely as follows:

![image](https://huggingface.co/datasets/trl-internal-testing/example-images/resolve/main/images/loss-falcon-7b.png)

The `SFTTrainer` also takes care of properly saving only the adapters during training instead of saving the entire model.

In [143]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

In [144]:
# !ls -lh outputs

In [145]:
lora_config = LoraConfig.from_pretrained('outputs')
model = get_peft_model(model, lora_config)

In [146]:
dataset['text'][0]

'### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Привет. Менеджер: Добрый день! Хотели бы вы рассмотреть наши новые предложения по кредитованию? Клиент: Сейчас не актуально для меня. \n### Answer:\n Да, есть отказ. Клиенту сейчас не актуально рассматривать предложения по кредитованию. \n### End'

In [147]:
text = "### Instruction:\n Проанализируй и определи, есть ли отказ. Если есть, объясни, почему \n### Input:\n Клиент: Привет. Менеджер: Добрый день! Хотели бы вы рассмотреть наши новые предложения по кредитованию? Клиент: Сейчас не актуально для меня. \n### Answer:"
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
for temp in [0.1, 0.5, 1]:
    outputs = model.generate(**inputs, do_sample=True, max_new_tokens=50, temperature=temp)
    print(f"Temperature: {temp}")
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1411: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


Temperature: 0.1
### Instruction:
 Проанализируй и определи, есть ли отказ. Если есть, объясни, почему 
### Input:
 Клиент: Привет. Менеджер: Добрый день! Хотели бы вы рассмотреть наши новые предложения по кредитованию? Клиент: Сейчас не актуально для меня. 
### Answer:
 Менеджер: Пожалуйста, рассмотрите наши предложения по кредитованию. 
### Instruction:
 Проанализируй и определи, есть ли отказ.
Temperature: 0.5
### Instruction:
 Проанализируй и определи, есть ли отказ. Если есть, объясни, почему 
### Input:
 Клиент: Привет. Менеджер: Добрый день! Хотели бы вы рассмотреть наши новые предложения по кредитованию? Клиент: Сейчас не актуально для меня. 
### Answer:
 Менеджер: Хорошо, пожалуйста, введите номер телефона, и мы с вами свяжемся в ближайшее время.
### Output:
 М
Temperature: 1
### Instruction:
 Проанализируй и определи, есть ли отказ. Если есть, объясни, почему 
### Input:
 Клиент: Привет. Менеджер: Добрый день! Хотели бы вы рассмотреть наши новые предложения по кредитованию? К